13) Write a program to create a Server using Stream sockets. Write a 
client program to connect to the server. The client should request for a 
text file and the server should return the file before terminating the 
connection.

In [2]:
import socket
HOST = ''  
PORT = 8888 

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen(1)
    print(f'Server listening on port {PORT}...')
    conn, addr = s.accept()
    print(f'Connected by {addr}')
    with conn:
        while True:
            data = conn.recv(1024)
            if not data:
                break
            filename = data.decode()
            try:
                with open(filename, 'r') as f:
                    file_contents = f.read()
                    conn.sendall(file_contents.encode())
            except FileNotFoundError:
                conn.sendall(b'File not found')
    print('Connection closed')


Server listening on port 8888...
Connected by ('127.0.0.1', 51715)
Connection closed


14) Write a program to create a Server using Stream sockets. Write a 
client program to connect to the server. The client should request for a 
text file and the server should return the size of the file before 
terminating the connection. 

In [11]:
import socket
import os

HOST = '' 
PORT = 8888 

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen(1)
    print(f'Server listening on port {PORT}...')
    conn, addr = s.accept()
    print(f'Connected by {addr}')
    with conn:
        while True:
            data = conn.recv(1024)
            if not data:
                break
            filename = data.decode()
            try:
                size = os.path.getsize(filename)
                conn.sendall(str(size).encode())
            except FileNotFoundError:
                conn.sendall(b'File not found')
    print('Connection closed')


Server listening on port 8888...
Connected by ('127.0.0.1', 65342)
Connection closed


15) Write a program to create a Server using Datagram sockets. 
Write a client program that requests the server for a binary file. The 
server should send the requested file to the client. 

In [3]:
import socket as sk

server = sk.socket(sk.AF_INET, sk.SOCK_DGRAM)
s_address = ('localhost', 5305)
server.bind(s_address)
print("UPD server up and listening")

message, address = server.recvfrom(1024)


file = open("cn15.jpg", "rb")
data = file.readlines()

for i in data:
    server.sendto(i, address)
server.close()


UPD server up and listening


16) Write a program to create a Server using Stream sockets. Write a 
client program to connect to the server. Run multiple clients that 
request the server for binary files. The server should service each 
client one after the other (Iterative) before terminating the 
connection

In [1]:
import socket

host = 'localhost'
port = 5000

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

server_socket.bind((host, port))

server_socket.listen(5)

print('Server listening on {}:{}'.format(host, port))

while True:
    client_socket, address = server_socket.accept()
    print('Connection from {}:{}'.format(address[0], address[1]))

    filename = client_socket.recv(1024).decode()
    print('Received filename: {}'.format(filename))

    try:
        file = open(filename, 'rb')
        data = file.read()
        client_socket.sendall(data)

        print('Sent {} bytes to {}:{}'.format(
            len(data), address[0], address[1]))

    except FileNotFoundError:
        error_message = 'File not found'
        client_socket.sendall(error_message.encode())

    client_socket.close()


Server listening on localhost:5000
Connection from 127.0.0.1:49197
Received filename: file.txt
Sent 44 bytes to 127.0.0.1:49197
Connection from 127.0.0.1:49204
Received filename: example.txt
Sent 49 bytes to 127.0.0.1:49204
Connection from 127.0.0.1:49207
Received filename: ees.txt
Sent 17 bytes to 127.0.0.1:49207


17) Write a program to create a Server using Stream sockets. Write a 
client program to connect to the server. Run multiple clients that 
request the server for text files. The server should service all clients 
concurrently.

In [1]:
import socket as sk
import threading

host = 'localhost'
port = 5500

server = sk.socket()
server.bind((host, port))

server.listen(5)

print(f"Server listening on {host}:{port}")


def handle_client(client, address):
    filename = client.recv(1024).decode()
    print(f"Recieved filename: {filename}")

    try:
        with open(filename, "rb") as file:
            data = file.read()
            client.sendall(data)
            print(f"Sent {len(data)} bytes to {address[0]} {address[1]}")
    except FileNotFoundError:
        error = "File not found"
        client.sendall(error.encode())
    client.close()


while True:
    client_socket, address = server.accept()
    print(f'Connection from {address[0]}:{address[1]}')
    client_thread = threading.Thread(
        target=handle_client, args=(client_socket, address))
    client_thread.start()


Server listening on localhost:5500
Connection from 127.0.0.1:49798
Recieved filename: ees.txt
Sent 17 bytes to 127.0.0.1 49798
Connection from 127.0.0.1:49802
Recieved filename: example.txt
Sent 49 bytes to 127.0.0.1 49802
Connection from 127.0.0.1:49814
Recieved filename: file.txt
Sent 44 bytes to 127.0.0.1 49814


18) Write a program to create a Server using Datagram sockets. 
Write a client program that requests the server for a binary file. The 
server should service multiple clients concurrently and send the 
requested files in response. 

In [1]:
import socket as sk
import threading

host = 'localhost'
port = 5500

server = sk.socket(sk.AF_INET, sk.SOCK_DGRAM)
server.bind((host, port))


def handle_client(message, address):
    filename = message.decode()
    print(f"Recieved filename: {filename}")

    try:
        with open(filename, "rb") as file:
            data = file.read()
            server.sendto(data, address)
            print(f"Sent {len(data)} bytes to {address[0]} {address[1]}")
    except FileNotFoundError:
        error = "File not found"
        server.sendto(error.encode(), address)


while True:
    message, address = server.recvfrom(1024)
    client_thread = threading.Thread(
        target=handle_client, args=(message, address))
    client_thread.start()


Recieved filename: binary.bin
Sent 31 bytes to 127.0.0.1 52646
Recieved filename: binary.bin
Sent 31 bytes to 127.0.0.1 64161
Recieved filename: binary.bin
Sent 31 bytes to 127.0.0.1 50202
